In [1]:
import pandas as pd
import numpy as np
import swifter

In [ ]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.at[index, 'WMA30']/df.at[index-1, 'WMA30']

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.at[index,'Close']/df.at[index-30, 'Close']

In [ ]:
def getData(path, comparePath = ""):
    try:
        df = pd.read_feather(path)
        if 'WMA30Slope' not in df.columns:
            dfCompare = [] if comparePath == "" else pd.read_feather(comparePath)
            if df['Close'].isnull().values.any():
                print(path + " close contains null")
            df['FiveYearHigh'] = df['Close'].rolling(window=260).max().shift(1).fillna(0)
            df['WMA30'] = df['Close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
            df['WMA30Slope'] = 0
            df['WMA30Slope'] = df.swifter.apply(lambda x: calculateSlopeOfTicker(df,(x.name)),axis=1)
            df['Volume'] = df['Volume'].replace(to_replace=0, method='ffill')
            df['Percent'] = 0
            df['Percent'] = df.swifter.apply(lambda x: product(df,(x.name)),axis=1)
            df = df.iloc[260:]
            if comparePath != "":
                df['RS'] = df['Percent'] - dfCompare['Percent']
            df.reset_index().to_feather(path)
    except:
        print(path)

In [ ]:
from turtle import st


stockList = pd.read_pickle("datasets/tickerList.pkl")
industryList = pd.read_pickle("datasets/industryList.pkl")
sectorList = pd.read_pickle("datasets/sectorList.pkl")
stockToSectorIndustryDict = pd.read_pickle('datasets/stockToSectorIndustry.pkl')

# getData("datasets/GSPC.feather")
for sector in sectorList:
    getData("datasets/sectorData/"+sector+".feather", "datasets/GSPC.feather")
for industry in industryList:
    getData("datasets/industriesData/"+industry[0]+"/"+industry[1]+".feather", "datasets/sectorData/"+sector+".feather")
for stock in set(stockList[0]):
    if stock not in stockToSectorIndustryDict.keys():
        continue
    getData("datasets/stockData/"+stock+".feather","datasets/industriesData/"+stockToSectorIndustryDict[stock][0]+"/"+stockToSectorIndustryDict[stock][1]+".feather")
